In [1]:
import pyspark
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
# Initialize pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .master('local[*]')\
                    .appName("Validation Split") \
                    .getOrCreate()

In [4]:
# Load dataframes

# path_to_df = 'gs://thesis_spotify_apc_bucket/df_data'
path_to_df = '/Volumes/T5/PROJECTS/U. THESIS/From GCP/df_data2'

df_tracks = spark.read.orc(path_to_df + '/df_tracks.orc')
df_playlists = spark.read.orc(path_to_df + '/df_playlists.orc')
df_playlists_metadata = spark.read.orc(path_to_df + '/df_playlists_metadata.orc')
df_challenge_playlists = spark.read.orc(path_to_df + '/df_challenge_playlists.orc')
df_challenge_playlists_metadata = spark.read.orc(path_to_df + '/df_challenge_playlists_metadata.orc')

In [6]:
# Group pid's based on num_tracks

from pyspark.sql.functions import collect_list

num_tracks = df_playlists_metadata.groupBy('num_tracks').agg(collect_list('pid').alias('pid')).orderBy('num_tracks', ascending=True)
# num_tracks.show()

In [7]:
# Get the number of test playlists with different values of num_tracks

from pyspark.sql.functions import desc

test_num_tracks_count = df_challenge_playlists_metadata.groupBy('num_tracks').count().sort(desc('count'))
# test_num_tracks_count.show()

In [8]:
%%time

# Randomly select pid's to be used as validation

from pyspark.sql.functions import col

validation_playlists = {}

for row in test_num_tracks_count.collect():
    i = row['num_tracks']
    j = row['count']
    pid_list = num_tracks.where(col('num_tracks') == i).select('pid').collect()
    try:
        validation_playlists[i] = np.random.choice(pid_list[0]['pid'], int(3*j), replace=False)
    except:
        print(len(pid_list[0]['pid']), i, j)
        validation_playlists[i] = []

CPU times: user 503 ms, sys: 192 ms, total: 696 ms
Wall time: 1min 36s


In [10]:
%%time

# Add and assign pid's to either validation 1 or validation 2

val1_playlist = {}
val2_playlist = {}
test_playlist = {}

for i in [0, 1, 5, 10, 25, 100]:
    
    val1_playlist[i] = []
    val2_playlist[i] = []
    test_playlist[i] = []
    
    value_counts = df_challenge_playlists_metadata.where(col('num_samples') == i)\
                                             .groupBy('num_tracks').count()

    for row in value_counts.collect():
        j = row['num_tracks']
        k = row['count']
        # print(len(validation_playlists[j]))
        val1_playlist[i] += list(validation_playlists[j][:k])
        validation_playlists[j] = validation_playlists[j][k:]
        val2_playlist[i] += list(validation_playlists[j][:k])
        validation_playlists[j] = validation_playlists[j][k:]
        test_playlist[i] += list(validation_playlists[j][:k])
        validation_playlists[j] = validation_playlists[j][k:]

CPU times: user 16.9 ms, sys: 6.77 ms, total: 23.7 ms
Wall time: 1.35 s


### Filtering
Validation logic:
- Each row in df_playlists is checked whether it belongs to val_playlist\[i\] AND its position is >= i
- df_train is made up of the other rows that do not belong to val1 or val2

Implementatin idea:

1. df_train = df_playlists

2. Filter df_train; remove the rows belong to val1_playlist\[0\] and val2_playlist\[0\] and place them to df_val1 and df_val2 respectively

3. Continue filtering df_train for i \[1, 5, 10, 25, 100\] and append them to df_val1 and df_val2

4. What's left in df_train is the rest of the rows that are not in val1 or val2

In [13]:
%%time

from pyspark.sql.functions import lit

# df_train = df_playlists

filter_val1 = [int(x) for x in val1_playlist[0]]
filter_val2 = [int(x) for x in val2_playlist[0]]
filter_test = [int(x) for x in test_playlist[0]]

# Append to df_val1 and df_val2
df_val1 = df_playlists.filter(df_playlists['pid'].isin(filter_val1))
df_val2 = df_playlists.filter(df_playlists['pid'].isin(filter_val2))
df_test_full = df_playlists.filter(df_playlists['pid'].isin(filter_test))
df_test_metadata = df_playlists_metadata.select('pid', 'name', 'num_tracks').filter(df_playlists_metadata['pid'].isin(filter_test))\
                                        .withColumn('num_samples', lit(0))

CPU times: user 505 ms, sys: 177 ms, total: 682 ms
Wall time: 3.85 s


In [14]:
%%time

for i in [1, 5, 10, 25, 100]:
    
    filter_val1 = [int(x) for x in val1_playlist[i]]
    filter_val2 = [int(x) for x in val2_playlist[i]]
    filter_test = [int(x) for x in test_playlist[i]]
    
    # Filter and append to df_val1 and df_val2
    append_to_val1 = df_playlists.filter((df_playlists['pid'].isin(filter_val1)) & (df_playlists['pos'] >= i))
    df_val1 = df_val1.union(append_to_val1)
    append_to_val2 = df_playlists.filter((df_playlists['pid'].isin(filter_val2)) & (df_playlists['pos'] >= i))
    df_val2 = df_val2.union(append_to_val2)
    append_to_test = df_playlists.filter(df_playlists['pid'].isin(filter_test))
    df_test_full = df_test_full.union(append_to_test)
    append_to_test_metadata = df_playlists_metadata.select('pid', 'name', 'num_tracks').filter(df_playlists_metadata['pid'].isin(filter_test))\
                                                   .withColumn('num_samples', lit(i))
    df_test_metadata = df_test_metadata.union(append_to_test_metadata)


CPU times: user 4.57 s, sys: 1.59 s, total: 6.17 s
Wall time: 31.2 s


In [15]:
%%time

# Remove rows that are in validation

df_train = df_playlists.join(df_val1, ['pid', 'tid', 'pos'], 'leftanti')\
                       .join(df_val2, ['pid', 'tid', 'pos'], 'leftanti')\
                       .join(df_test_full, ['pid', 'tid', 'pos'], 'leftanti')

CPU times: user 2.28 ms, sys: 1.46 ms, total: 3.74 ms
Wall time: 1.31 s


In [16]:
val1_pids = np.hstack([val1_playlist[i] for i in val1_playlist])
val2_pids = np.hstack([val2_playlist[i] for i in val2_playlist])
test_pids = np.hstack([test_playlist[i] for i in test_playlist])

In [17]:
# Checking
print(len(val1_pids), len(val2_pids), len(test_pids))

10000 10000 10000


In [19]:
# Set up dataframes for testing

temp_test = df_test_full.join(df_test_metadata.select('pid', 'num_samples'), ['pid'], 'left')

# For playlists with num_samples = 1, take only the first 1 tracks
df_test_playlists = temp_test.filter((col('num_samples') == 1) & (col('pos') < 1))

# For playlists with num_samples = 5, take only the first 5 tracks
df_test_playlists = df_test_playlists.union(temp_test.filter((col('num_samples') == 5) & (col('pos') < 5)))

# For playlists with num_samples = 10, take only the first 10 tracks
df_test_playlists = df_test_playlists.union(temp_test.filter((col('num_samples') == 10) & (col('pos') < 10)))

In [26]:
# For playlists with num_samples = 10, 25, and 100, half of them have their tracks picked at random and the other half picked in order

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# random_pids = {}
# ordered_pids = {}

for i in [25, 100]:
    random_pids = [int(x) for x in np.random.choice(test_playlist[i], 1000, replace=False)]
    ordered_pids = [int(x) for x in list(set(test_playlist[i]) - set(random_pids))]
    
    df_test_playlists = df_test_playlists.union(temp_test.filter((col('pid').isin(ordered_pids)) & (col('pos') < i)))
    chosen_random = temp_test.filter(col('pid').isin(random_pids))
    pd_chosen_random = chosen_random.toPandas().groupby('pid').sample(n=i, random_state=1)
    chosen_random = spark.createDataFrame(pd_chosen_random)
    df_test_playlists = df_test_playlists.union(chosen_random)

In [21]:
# Concat df_train with df_test_playlists

df_train_final = df_train.union(df_test_playlists.select(['pid', 'tid', 'pos']))  # select to make sure the order of columns are the same

In [23]:
df_train_final.describe().show()

+-------+------------------+-----------------+------------------+
|summary|               pid|              tid|               pos|
+-------+------------------+-----------------+------------------+
|  count|          11210327|         11210327|          11210327|
|   mean|170987.08450690153|518038.5394889016|49.463758015265746|
| stddev| 72343.57508834456|113800.6593490638|44.754728705095445|
|    min|                 0|                0|                 0|
|    max|            278999|           995381|               249|
+-------+------------------+-----------------+------------------+



In [25]:
# Save dataframes as ORC files

# path_to_write = 'gs://thesis_spotify_apc_bucket/df_data'
path_to_write = '/Volumes/T5/PROJECTS/U. THESIS/Dataset/new_df_data'

df_train_final.write.mode('overwrite').orc(path_to_write + '/df_train.orc')
df_val1.write.mode('overwrite').format('orc').save(path_to_write + '/df_val1.orc')
df_val2.write.mode('overwrite').format('orc').save(path_to_write + '/df_val2.orc')
df_test_full.write.mode('overwrite').format('orc').save(path_to_write + '/df_test_full.orc')
df_test_playlists.write.mode('overwrite').format('orc').save(path_to_write + '/df_test_playlists.orc')
df_test_metadata.write.mode('overwrite').format('orc').save(path_to_write + '/df_test_metadata.orc')

In [26]:
import joblib

# !mkdir to_save

joblib.dump(val1_pids, path_to_write + '/val1_pids.pkl')
joblib.dump(val2_pids, path_to_write + '/val2_pids.pkl')
joblib.dump(test_pids, path_to_write + '/test_pids.pkl')

['to_save/test_pids.pkl']

In [27]:
# Upload file to GCS

# ! gsutil cp to_save/* gs://thesis_spotify_apc_bucket/df_data

Copying file://to_save/test_pids.pkl [Content-Type=application/octet-stream]...
Copying file://to_save/val1_pids.pkl [Content-Type=application/octet-stream]... 
Copying file://to_save/val2_pids.pkl [Content-Type=application/octet-stream]... 
/ [3 files][234.9 KiB/234.9 KiB]                                                
Operation completed over 3 objects/234.9 KiB.                                    


In [29]:
# Save dataframes as HDF files (CHECKING)

import pandas as pd

train = df_train_final.toPandas()
val1 = df_val1.toPandas()
val2 = df_val2.toPandas()

train.to_hdf(path_to_write + '/df_train.hdf', key='abc')
val1.to_hdf(path_to_write + '/df_val1.hdf', key='abc')
val2.to_hdf(path_to_write + '/df_val2.hdf', key='abc')

In [30]:
# ! gsutil cp df_data/df_train.hdf gs://thesis_spotify_apc_bucket/df_data/df_train.hdf
# ! gsutil cp df_data/df_val1.hdf gs://thesis_spotify_apc_bucket/df_data/df_val1.hdf
# ! gsutil cp df_data/df_val2.hdf gs://thesis_spotify_apc_bucket/df_data/df_val2.hdf

Copying file://df_data/df_train.hdf [Content-Type=application/x-hdf]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][342.1 MiB/342.1 MiB]                                                
Operation completed over 1 objects/342.1 MiB.                                    
Copying file://df_data/df_val1.hdf [Content-Type=application/x-hdf]...
/ [1 fil

In [4]:
# # Ketinggalan save

# path_to_write = 'gs://thesis_spotify_apc_bucket/df_data'

# df_test_full = spark.read.orc(path_to_write + '/df_test_full.orc')
# df_test_playlists = spark.read.orc(path_to_write + '/df_test_playlists.orc')
# df_test_metadata = spark.read.orc(path_to_write + '/df_test_metadata.orc')

In [6]:
# import pandas as pd

# spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# test_full = df_test_full.toPandas()
# test_playlists = df_test_playlists.toPandas()
# test_metadata = df_test_metadata.toPandas()

# test_full.to_hdf('df_data/df_test_full.hdf', key='abc')
# test_playlists.to_hdf('df_data/df_test_playlists.hdf', key='abc')
# test_metadata.to_hdf('df_data/df_test_metadata.hdf', key='abc')

In [7]:
# ! gsutil cp df_data/df_test_full.hdf gs://thesis_spotify_apc_bucket/df_data/df_test_full.hdf
# ! gsutil cp df_data/df_test_playlists.hdf gs://thesis_spotify_apc_bucket/df_data/df_test_playlists.hdf
# ! gsutil cp df_data/df_test_metadata.hdf gs://thesis_spotify_apc_bucket/df_data/df_test_metadata.hdf

Copying file://df_data/df_test_full.hdf [Content-Type=application/x-hdf]...
/ [1 files][ 29.9 MiB/ 29.9 MiB]                                                
Operation completed over 1 objects/29.9 MiB.                                     
Copying file://df_data/df_test_playlists.hdf [Content-Type=application/x-hdf]...
/ [1 files][  9.7 MiB/  9.7 MiB]                                                
Operation completed over 1 objects/9.7 MiB.                                      
Copying file://df_data/df_test_metadata.hdf [Content-Type=application/x-hdf]...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      
